### MODEL.PY

In [ ]:
import os
import csv
import cv2
import numpy as np
import sklearn
import math
from random import shuffle
from PIL import Image

samples = []
with open('../../opt/carnd_p3/data/driving_log.csv') as csvfile:
# with open('../data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
    print('csv file read successfully')
    print(samples[1440])


from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)
def process_image(img):
    #do no preprocessing for now
    return img

def generator(samples, batch_size=32):
    num_samples = len(samples)
    #print('sample size = ', num_samples)
    
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        steer_correction = 0.2

            
        for offset in range(1, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            
            #batch_samples[np.isnan(batch_samples)]=0
            

            images = []
            angles = []
            #print('batch sample size',len(batch_samples))
            
            for batch_sample in batch_samples:
#                print('Batch sample vlaue = ',batch_sample[3])
                steering_center = float(batch_sample[3])
                steering_left = steering_center + steer_correction
                steering_right = steering_center - steer_correction
            
                #source_path = batch_sample[0] # loading center image only
                #filename = source_path.split('/')[-1]
                #print('filename=',filename)
                current_path = '../../opt/carnd_p3/data/IMG/'
                #print ('current path = ',current_path)
                img_center = process_image(np.asarray(Image.open(current_path + batch_sample[0].split('/')[-1])))
                img_left = process_image(np.asarray(Image.open(current_path + batch_sample[1].split('/')[-1])))
                img_right = process_image(np.asarray(Image.open(current_path + batch_sample[2].split('/')[-1])))
            
                images.extend([img_center, img_left, img_right])
                angles.extend([steering_center, steering_left, steering_right])
            #print('total images for this batch = ', np.array(images).size)
            #print('total angles for this batch = ', np.array(angles).size)
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

# Set our batch size
batch_size=32

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=batch_size)
validation_generator = generator(validation_samples, batch_size=batch_size)



from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers import Conv2D, Cropping2D
#from keras.layers.pooling

model = Sequential()
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))
model.add(Lambda(lambda x:x/255-0.5))
print('model shape after lambda = ', model.output_shape)
#for layer in model.layers:
#    print(layer.output_shape)
model.add(Conv2D(24,5,5,subsample=(2,2),activation='relu'))
model.add(Conv2D(36,5,5,subsample=(2,2),activation='relu'))
model.add(Conv2D(48,5,5,subsample=(2,2),activation='relu'))
model.add(Conv2D(64,3,3,activation='relu'))
model.add(Conv2D(64,3,3,activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))
print('model shape after dense = ',model.output_shape)

model.compile(loss='mse',optimizer='adam')
#model.fit(X_train, y_train, validation_split = 0.25, shuffle = True, nb_epoch=7)
model.fit_generator(train_generator, steps_per_epoch=math.ceil(len(train_samples)/batch_size)*3, validation_data=validation_generator, validation_steps=math.ceil(len(validation_samples)/batch_size)*3, epochs=5, verbose=1)

model.save('model_01.h5')




# ERROR that I am getting

In [ ]:
(/opt/carnd_p3/behavioral) root@e2c06358fe1e:/home/workspace# python model.py
csv file read successfully
['IMG/center_2016_12_01_13_35_04_080.jpg', ' IMG/left_2016_12_01_13_35_04_080.jpg', ' IMG/right_2016_12_01_13_35_04_080.jpg'
, ' 0.0904655', ' 0.9855326', ' 0', ' 30.18644']
Using TensorFlow backend.
model shape after lambda =  (None, 90, 320, 3)
model.py:88: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), strides=(2, 2), activation="relu
")`
  model.add(Conv2D(24,5,5,subsample=(2,2),activation='relu'))
model.py:89: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), strides=(2, 2), activation="relu
")`
  model.add(Conv2D(36,5,5,subsample=(2,2),activation='relu'))
model.py:90: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), strides=(2, 2), activation="relu")`
  model.add(Conv2D(48,5,5,subsample=(2,2),activation='relu'))
model.py:91: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  model.add(Conv2D(64,3,3,activation='relu'))
model.py:92: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  model.add(Conv2D(64,3,3,activation='relu'))
model shape after dense =  (None, 1)
Epoch 1/5
2019-07-15 18:11:28.303843: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2019-07-15 18:11:28.303906: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2019-07-15 18:11:28.303918: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2019-07-15 18:11:28.303927: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
2019-07-15 18:11:28.303935: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use FMA instructions, but these are available on your machine and could speed up CPU computations.
2019-07-15 18:11:28.408961: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:893] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-07-15 18:11:28.409710: I tensorflow/core/common_runtime/gpu/gpu_device.cc:955] Found device 0 with properties: 
name: Tesla K80
major: 3 minor: 7 memoryClockRate (GHz) 0.8235
pciBusID 0000:00:04.0
Total memory: 11.17GiB
Free memory: 11.09GiB
2019-07-15 18:11:28.409769: I tensorflow/core/common_runtime/gpu/gpu_device.cc:976] DMA: 0 
2019-07-15 18:11:28.409796: I tensorflow/core/common_runtime/gpu/gpu_device.cc:986] 0:   Y 
2019-07-15 18:11:28.409823: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1045] Creating TensorFlow device (/gpu:0) -> (device: 0, name: Tesla K80, pci bus id: 0000:00:04.0)
164/603 [=======>......................] - ETA: 2:38 - loss: 0.0235Traceback (most recent call last):
  File "model.py", line 102, in <module>
    model.fit_generator(train_generator, steps_per_epoch=math.ceil(len(train_samples)/batch_size)*3, validation_data=validation_generator, validation_steps=math.ceil(len(validation_samples)/batch_size)*3, epochs=5, verbose=1)
  File "/opt/carnd_p3/behavioral/lib/python3.5/site-packages/keras/legacy/interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "/opt/carnd_p3/behavioral/lib/python3.5/site-packages/keras/engine/training.py", line 1418, in fit_generator
    initial_epoch=initial_epoch)
  File "/opt/carnd_p3/behavioral/lib/python3.5/site-packages/keras/engine/training_generator.py", line 181, in fit_generator
    generator_output = next(output_generator)
  File "/opt/carnd_p3/behavioral/lib/python3.5/site-packages/keras/utils/data_utils.py", line 709, in get
    six.reraise(*sys.exc_info())
  File "/opt/carnd_p3/behavioral/lib/python3.5/site-packages/six.py", line 693, in reraise
    raise value
  File "/opt/carnd_p3/behavioral/lib/python3.5/site-packages/keras/utils/data_utils.py", line 685, in get
    inputs = self.queue.get(block=True).get()
  File "/opt/carnd_p3/behavioral/lib/python3.5/multiprocessing/pool.py", line 608, in get
    raise self._value
  File "/opt/carnd_p3/behavioral/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/opt/carnd_p3/behavioral/lib/python3.5/site-packages/keras/utils/data_utils.py", line 626, in next_sample
    return six.next(_SHARED_SEQUENCES[uid])
  File "model.py", line 47, in generator
    steering_center = float(batch_sample[3])
ValueError: could not convert string to float: 'steering'